<a href="https://colab.research.google.com/github/Mario-16180/BEDU-M4-DataAnalysisProject-CarAccidents/blob/main/BEDU_DataAnalysis_CarAccidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd